# Initiation au scraping

Objectif récupérer des informations disponibles sur le web , mais à priori que pour des humains, de manière programmatique et automatisée (voire périodique).

Dans l'ordre:

1. Requête http
2. Extraction de texte
3. Parser du html
4. Nettoyage des données
5. Structuration du dataset

## Requête http

In [1]:
from requests import get

**OBJECTIF** récupérer le contenu de la table se trouvant à [cette adresse](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) et contenant les informations sur les 500 plus grosses capitalisation US.

In [2]:
ADRESSE = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [3]:
requete = get(ADRESSE)

In [4]:
type(requete)

requests.models.Response

In [5]:
dir(requete)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Dans l'objectif d'avoir un script final robuste, il faut pouvoir intercepter les erreurs :

In [6]:
# 200 : OK
requete.status_code

200

On consultera [cette page](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) pour les différents codes possibles.

In [7]:
# ATTENTION au type du contenu
type(requete.content)

bytes

In [8]:
requete.apparent_encoding

'utf-8'

In [9]:
type(requete.text)

str

In [10]:
# Récupération du code source
PAGE = requete.text

In [12]:
len(PAGE)

741512

In [13]:
len(PAGE.splitlines())

9739

**CONCLUSION** le code source de la page web est relativement conséquent.

**EXERCICE** récupérer la partie de la page, contenant juste le code source concernant la table qui nous intéresse.

In [15]:
debut = 0
indices = list()
while debut >= 0:
    debut = PAGE.find("<table class=", debut+1)
    indices.append(debut)

In [16]:
indices

[45233, 259667, -1]

On voit deux éléments `table` dans la page

In [17]:
premier, deuxieme, _ = indices

In [18]:
print(PAGE[premier:premier+200])

<table class="wikitable sortable sticky-header" id="constituents">

<tbody><tr>
<th><a href="/wiki/Ticker_symbol" title="Ticker symbol">Symbol</a>
</th>
<th>Security</th>
<th><a href="/wiki/Global_Ind


In [19]:
print(PAGE[deuxieme:deuxieme+200])

<table class="wikitable sortable" id="changes">

<tbody><tr>
<th data-sort-type="date" rowspan="2">Date
</th>
<th colspan="2">Added
</th>
<th colspan="2">Removed
</th>
<th rowspan="2">Reason
</th></tr


L'attribut `id` permet de distinguer les deux, car celle qu'on cherche est d'attribut `constituents`.

**ATTENTION** on peut voir que les attributs de `class` ne sont pas exactement les mêmes que dans les outils de développement du navigateur. 
Ici la différence vient du fait que du javascript a été exécuté par le navigateur!

Ici l'attribut supplémentaire est `jquery-tablesorter` qui permet dynamiquement de réordonner les éléments de la table suivant une colonne.
Les données recherchées sont quand même bien présentes dans le html qu'on a.

In [21]:
fin = PAGE.find("</table>", premier) + len("</table>")

In [23]:
code_table = PAGE[premier:fin]
debut, *_, fin = code_table.splitlines()

In [26]:
assert debut.startswith("<table class=")
print(debut)

<table class="wikitable sortable sticky-header" id="constituents">


In [30]:
assert fin.endswith("</table>")
print(repr(fin))

'</td></tr></tbody></table>'


**EXERCICE** 
1. Récupérer la liste des code sources des lignes de la table
2. Créer une dataclass `Ligne` représentant une ligne structurée
3. Récupérer une liste de `Ligne`

In [31]:
from dataclasses import dataclass

In [32]:
from typing import Iterable

In [34]:
def decoupe_lignes(page: str, debut: int = 0) -> Iterable[str]:
    debut = page.find("<tr>", debut)
    if debut != -1:
        fin = page.find("</tr>", debut) + len("</tr>")
        yield page[debut:fin]
        yield from decoupe_lignes(page, fin)
        

In [35]:
test_lignes = """
blabla
<tr>première</tr>
blablabla
blablabla
<tr> deuxième </tr>
blabla
<tr> fin </tr>
blabla
"""

In [36]:
list(decoupe_lignes(test_lignes))

['<tr>première</tr>', '<tr> deuxième </tr>', '<tr> fin </tr>']

In [38]:
HEADER, *CODE_LIGNES = list(decoupe_lignes(code_table))

In [40]:
print(HEADER)

<tr>
<th><a href="/wiki/Ticker_symbol" title="Ticker symbol">Symbol</a>
</th>
<th>Security</th>
<th><a href="/wiki/Global_Industry_Classification_Standard" title="Global Industry Classification Standard">GICS</a> Sector</th>
<th>GICS Sub-Industry</th>
<th>Headquarters Location</th>
<th>Date added</th>
<th><a href="/wiki/Central_Index_Key" title="Central Index Key">CIK</a></th>
<th>Founded
</th></tr>


In [43]:
@dataclass
class Ligne:
    symbole: str
    lien_nyse: str
    securite: str
    lien_wiki: str
    gics: str # Global Industry Classification Standard
    sous_gics: str
    localisation: str
    date_entree: str
    cik: str # Central Index Key
    date_fondation: str

In [65]:
ligne_test = CODE_LIGNES[0].replace("\n", " ")
print(ligne_test)

<tr> <td><a rel="nofollow" class="external text" href="https://www.nyse.com/quote/XNYS:MMM">MMM</a> </td> <td><a href="/wiki/3M" title="3M">3M</a></td> <td>Industrials</td> <td>Industrial Conglomerates</td> <td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td> <td>1957-03-04</td> <td>0000066740</td> <td>1902 </td></tr>


In [67]:
td = re.compile("<td>(.*?)</td>")

In [68]:
td.findall(ligne_test)

['<a rel="nofollow" class="external text" href="https://www.nyse.com/quote/XNYS:MMM">MMM</a> ',
 '<a href="/wiki/3M" title="3M">3M</a>',
 'Industrials',
 'Industrial Conglomerates',
 '<a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a>',
 '1957-03-04',
 '0000066740',
 '1902 ']

In [76]:
def genere_ligne(ligne: str) -> Ligne:
    ligne = ligne.replace("\n", " ")
    
    td = re.compile("<td>(.*?)</td>")
    symbole, securite, gics, sgics, loc, intro, cik, fond = td.findall(ligne)
    
    motif_lien_texte = re.compile("""<a.*?href="(.*?)".*?>(.*?)</a>""")
    lien_nyse, symbole = motif_lien_texte.match(symbole).groups()
    lien_wiki, securite = motif_lien_texte.match(securite).groups()
    _, localisation = motif_lien_texte.match(loc).groups()
    return Ligne(
        symbole=symbole,
        lien_nyse=lien_nyse,
        securite=securite,
        lien_wiki="https://en.wikipedia.org/" + lien_wiki,
        gics=gics,
        sous_gics=sgics,
        localisation=localisation,
        date_entree=intro,
        cik=cik,
        date_fondation=fond.strip(),
    )

In [77]:
print(genere_ligne(ligne_test))

Ligne(symbole='MMM', lien_nyse='https://www.nyse.com/quote/XNYS:MMM', securite='3M', lien_wiki='https://en.wikipedia.org//wiki/3M', gics='Industrials', sous_gics='Industrial Conglomerates', localisation='Saint Paul, Minnesota', date_entree='1957-03-04', cik='0000066740', date_fondation='1902')


In [78]:
# assemblage du tout
lignes = [
    genere_ligne(ligne)
    for ligne in CODE_LIGNES
]

In [79]:
len(lignes)

503

In [80]:
from random import randint

In [81]:
lignes[randint(0, 502)]

Ligne(symbole='ABBV', lien_nyse='https://www.nyse.com/quote/XNYS:ABBV', securite='AbbVie', lien_wiki='https://en.wikipedia.org//wiki/AbbVie', gics='Health Care', sous_gics='Biotechnology', localisation='North Chicago, Illinois', date_entree='2012-12-31', cik='0001551152', date_fondation='2013 (1888)')

**EXERCICE** sérialiser le résultat sur disque au format json.

In [82]:
import json

In [86]:
ligne0 = lignes[0]

In [89]:
ligne0.__dict__

{'symbole': 'MMM',
 'lien_nyse': 'https://www.nyse.com/quote/XNYS:MMM',
 'securite': '3M',
 'lien_wiki': 'https://en.wikipedia.org//wiki/3M',
 'gics': 'Industrials',
 'sous_gics': 'Industrial Conglomerates',
 'localisation': 'Saint Paul, Minnesota',
 'date_entree': '1957-03-04',
 'cik': '0000066740',
 'date_fondation': '1902'}

In [91]:
mon_json = json.dumps([ligne.__dict__ for ligne in lignes])

In [93]:
deserialises = [Ligne(**dico) for dico in json.loads(mon_json)]

In [94]:
deserialises

[Ligne(symbole='MMM', lien_nyse='https://www.nyse.com/quote/XNYS:MMM', securite='3M', lien_wiki='https://en.wikipedia.org//wiki/3M', gics='Industrials', sous_gics='Industrial Conglomerates', localisation='Saint Paul, Minnesota', date_entree='1957-03-04', cik='0000066740', date_fondation='1902'),
 Ligne(symbole='AOS', lien_nyse='https://www.nyse.com/quote/XNYS:AOS', securite='A. O. Smith', lien_wiki='https://en.wikipedia.org//wiki/A._O._Smith', gics='Industrials', sous_gics='Building Products', localisation='Milwaukee, Wisconsin', date_entree='2017-07-26', cik='0000091142', date_fondation='1916'),
 Ligne(symbole='ABT', lien_nyse='https://www.nyse.com/quote/XNYS:ABT', securite='Abbott Laboratories', lien_wiki='https://en.wikipedia.org//wiki/Abbott_Laboratories', gics='Health Care', sous_gics='Health Care Equipment', localisation='North Chicago, Illinois', date_entree='1957-03-04', cik='0000001800', date_fondation='1888'),
 Ligne(symbole='ABBV', lien_nyse='https://www.nyse.com/quote/XNYS:

## Expressions régulières

Langage permettant de décrire des motifs permettant d'identifier des chaines de caractères.

On consultera [la documentation](https://docs.python.org/3/library/re.html) pour une description plus complète de la syntaxe.

In [44]:
import re

On va décrire une chaine
1. Commençant par au moins un `a`
2. Suivi d'un chiffre de 0 à 5
3. suivi d'un caractère `b` répété au plus trois fois
4. se finissant par un chiffre de 3 à 9

In [46]:
motif = re.compile(r"^aa*[0-5]b{0,3}[3-9]$")

In [47]:
motif.match("a3bb8")

<re.Match object; span=(0, 5), match='a3bb8'>

In [48]:
motif.match("aa0bbb3")

<re.Match object; span=(0, 7), match='aa0bbb3'>

In [49]:
motif.match("0b3")

In [50]:
motif.match("a6b3")

In [51]:
motif.match("ab5b3")

In [52]:
motif.match("aa3bbbb3")

In [53]:
motif.match("aa3bbb2")

On peut extraire des sous parties intéressantes de la chaine.

In [55]:
motif2 = re.compile(r"^(aa*)[0-5](b{0,3})[3-9]$")

In [57]:
resultat = motif2.match("aa0bbb3")

In [58]:
type(resultat)

re.Match

In [59]:
resultat.groups()

('aa', 'bbb')

On peut aussi rechercher les apparitions d'un motif dans une chaine de caractères.

In [60]:
nombre = re.compile("[1-9][0-9]*")

In [62]:
nombre.findall("123 blabla 456 bla012bla 12")

['123', '456', '12', '12']